In [66]:
__author__ = 'Chien-Hsiu Lee <chien-hsiu.lee@noirlab.edu>'
__version__ = '20210507' # yyyymmdd
__datasets__ = ['']
__keywords__ = ['ANTARES', 'Transient Name Server']

# Interfacing ANTARES with Transient Name Server

**This notebook requires the installation of ANTARES client (https://noao.gitlab.io/antares/client/). For more detail on programmatic access of TNS via API, please see the help page of TNS (https://www.wis-tns.org/content/tns-getting-started).**



Transient Name Server (TNS) is the official IAU
mechanism for reporting new astronomical transients (ATs). We can report filtered ANTARES locus of certain type or candidate objects with the TNS API as follows.


### Table of contents
* [Imports & setup](#import)
* [Creating a report from an ANTARES locus](#report)
* [Bulk report a list of ANTARES loci](#bulkreport)


<a class="anchor" id="import"></a>
# Imports & Setup
The first step is to import TNS API. This is based on the sample python code in https://www.wis-tns.org/sites/default/files/api/tns_api_bulk_report.py.zip . Please also create a directory json_reports_for_sending/ under the current path. The reports will be stored under this directory. 

In this notebook it will first test the report against TNS sandbox (TNS="sandbox.wis-tns.org"). For offical reports, please switch the url to (TNS="www.wis-tns.org"). The "api_key" is for ANTARES_bot; please keep it confidential.


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on December 2020

Developed and tested on:

- Linux 18.04 LTS
- Windows 10
- Python 3.7 (Spyder)

@author: Nikola Knezevic ASTRO DATA
"""

import os
import requests
import json
from collections import OrderedDict
import time    
import sys


###########################################################################################
####################################### PARAMETERS ########################################

#TNS="www.wis-tns.org"
TNS="sandbox.wis-tns.org"
url_tns_api="https://"+TNS+"/api"

api_key="916xxxx"
list_of_filenames="Here put your list of filenames for uploading."
report_filename="Here put your report filename."
# report type can only be "tsv" or "json"
report_type="Here put the type of your report."
id_report="Here put your report ID for getting report's reply."

# current working directory
cwd=os.getcwd()
# folder containing files for uploading
upload_folder=os.path.join(cwd,'files_for_uploading')
# folder containing tsv reports for sending
tsv_reports_folder=os.path.join(cwd,'tsv_reports_for_sending')
# folder containing json reports for sending
json_reports_folder=os.path.join(cwd,'json_reports_for_sending')

# http errors
http_errors={                       
304: 'Error 304: Not Modified: There was no new data to return.',
400: 'Error 400: Bad Request: The request was invalid. '\
     'An accompanying error message will explain why.',
403: 'Error 403: Forbidden: The request is understood, but it has '\
     'been refused. An accompanying error message will explain why.',
404: 'Error 404: Not Found: The URI requested is invalid or the '\
     'resource requested, such as a category, does not exists.',
500: 'Error 500: Internal Server Error: Something is broken.',
503: 'Error 503: Service Unavailable.'
}       

# how many second to sleep
SLEEP_SEC=1
# max number of time to check response
LOOP_COUNTER=60
# keeping sys.stdout
old_stdout=sys.stdout

###########################################################################################
###########################################################################################


###########################################################################################
######################################## FUNCTIONS ########################################

# function for changing data to json format
def format_to_json(source):
    # change data to json format and return
    parsed=json.loads(source,object_pairs_hook=OrderedDict)
    result=json.dumps(parsed,indent=4)
    return result

# function for uploading files trough api
def upload_files(url,list_of_files):
  try:
    # url for uploading files
    upload_url=url+'/file-upload'
    # api key data
    api_data={'api_key':api_key}
    # construct a dictionary of files and their data
    files_data={}
    for i in range(len(list_of_files)):
        file_name=list_of_files[i]
        file_path=os.path.join(upload_folder,file_name)
        key='files['+str(i)+']'     
        if file_name.lower().endswith(('.asci', '.ascii')):
            value=(file_name, open(file_path), 'text/plain')
        else:
            value=(file_name, open(file_path,'rb'), 'application/fits')
        files_data[key]=value
    # upload all files using request module
    response=requests.post(upload_url, data=api_data, files=files_data)
    # return response
    return response
  except Exception as e:
    return [None,'Error message : \n'+str(e)]

# function for sending tsv reports (AT or Classification)
def send_tsv_report(url,tsv_report):
  try:
    # url for sending tsv reports
    tsv_url=url+'/csv-report'
    # api key data 
    api_data={'api_key':api_key}
    # tsv report file path
    tsv_file_path=os.path.join(tsv_reports_folder,tsv_report)
    # read tsv data from file
    tsv_read=(tsv_report, open(tsv_file_path,'rb'))
    # construct a dictionary of tsv data
    tsv_data={'csv':tsv_read}
    # send tsv report using request module
    response=requests.post(tsv_url, data=api_data, files=tsv_data)
    # return response
    return response
  except Exception as e:
    return [None,'Error message : \n'+str(e)]

# function for sending json reports (AT or Classification)
def send_json_report(url,json_report):
  try:
    # url for sending json reports
    json_url=url+'/bulk-report'
    # json report file path
    json_file_path=os.path.join(json_reports_folder,json_report)
    # read json data from file
    json_read=format_to_json(open(json_file_path).read())
    # construct a dictionary of api key data and json data
    json_data={'api_key':api_key, 'data':json_read}
    # send json report using request module
    response=requests.post(json_url, data=json_data)
    # return response
    return response
  except Exception as e:
    return [None,'Error message : \n'+str(e)]

# function for getting reply from report
def reply(url, report_id):
  try:
    # url for getting report reply
    reply_url=url+'/bulk-report-reply'
    # construct a dictionary of api key data and report id
    reply_data={'api_key':api_key, 'report_id':report_id}
    # send report ID using request module
    response=requests.post(reply_url, data=reply_data)
    # return response
    return response
  except Exception as e:
    return [None,'Error message : \n'+str(e)]

# function that checks response and
# returns True if everything went OK
# or returns False if something went wrong
def check_response(response):
    # if response exists
    if None not in response:
        # take status code of that response
        status_code=int(response.status_code)
        if status_code==200:
            # response as json data
            json_data=response.json()
            # id code
            id_code=str(json_data['id_code'])
            # id message
            id_message=str(json_data['id_message'])
            # print id code and id message
            print ("ID code = "+id_code)
            print ("ID message = "+id_message)
            # check if id code is 200 and id message OK
            if (id_code=="200" and id_message=="OK"):
                return True
            #special case
            elif (id_code=="400" and id_message=="Bad request"):
                return None
            else:
                return False
        else:
            # if status code is not 200, check if it exists in
            # http errors
            if status_code in list(http_errors.keys()):
                print (list(http_errors.values())
                       [list(http_errors.keys()).index(status_code)])
            else:
                print ("Undocumented error.")
            return False
    else:
        # response doesn't exists, print error
        print (response[1])
        return False

# find all occurrences of a specified key in json data
# and return all values for that key
def find_keys(key, json_data):
    if isinstance(json_data, list):
        for i in json_data:
            for x in find_keys(key, i):
               yield x
    elif isinstance(json_data, dict):
        if key in json_data:
            yield json_data[key]
        for j in list(json_data.values()):
            for x in find_keys(key, j):
                yield x

# print feedback
def print_feedback(json_feedback):
    # find all message id-s in feedback
    message_id=list(find_keys('message_id',json_feedback))
    # find all messages in feedback
    message=list(find_keys('message',json_feedback))
    # find all obj names in feedback
    objname=list(find_keys('objname',json_feedback))
    # find all new obj types in feedback
    new_object_type=list(find_keys('new_object_type',json_feedback))
    # find all new obj names in feedback
    new_object_name=list(find_keys('new_object_name',json_feedback))
    # find all new redshifts in feedback
    new_redshift=list(find_keys('new_redshift',json_feedback))
    # index counters for objname, new_object_type, new_object_name
    # and new_redshift lists
    n_o=0
    n_not=0
    n_non=0
    n_nr=0
    # messages to print
    msg=[]
    # go trough every message and print
    for j in range(len(message)):
        m=str(message[j])
        m_id=str(message_id[j])
        if m_id not in ['102','103','110']:
            if m.endswith('.')==False:
                m=m+'.'
            if m_id=='100' or  m_id=='101':
                m="Message = "+m+" Object name = "+str(objname[n_o])
                n_o=n_o+1
            elif m_id=='120': 
                m="Message = "+m+" New object type = "+str(new_object_type[n_not])
                n_not=n_not+1
            elif m_id=='121':
                m="Message = "+m+" New object name = "+str(new_object_name[n_non])
                n_non=n_non+1
            elif m_id=='122' or  m_id=='123':
                m="Message = "+m+" New redshift = "+str(new_redshift[n_nr])
                n_nr=n_nr+1
            else:
                m="Message = "+m
            msg.append(["Message ID = "+m_id,m])
    # return messages       
    return msg

# sending report id to get reply of the report
# and printing that reply
def print_reply(url,report_id):
    # sending reply using report id and checking response
    print ("Sending reply for the report id "+report_id+" ...")
    reply_res=reply(url, report_id)
    reply_res_check=check_response(reply_res)
    # if reply is sent
    if reply_res_check==True:
        print ("The report was successfully processed on the TNS.\n")
        # reply response as json data
        json_data=reply_res.json()
        # feedback of the response
        feedback=list(find_keys('feedback',json_data))
        # check if feedback is dict or list
        if type(feedback[0])==type([]):
            feedback=feedback[0]
        # go trough feedback
        for i in range(len(feedback)):
            # feedback as json data
            json_f=feedback[i]
            # feedback keys
            feedback_keys=list(json_f.keys())
            # messages for printing
            msg=[]
            # go trough feedback keys
            for j in range(len(feedback_keys)):
                key=feedback_keys[j]
                json_feed=json_f[key]
                msg=msg+print_feedback(json_feed)
            if msg!=[]:
                print ("-----------------------------------"\
                       "-----------------------------------" )
                for k in range(len(msg)):
                    print (msg[k][0])
                    print (msg[k][1])
                print ("-----------------------------------"\
                       "-----------------------------------\n") 
    else:
        if (reply_res_check!=None):
            print ("The report doesn't exist on the TNS.")
        else:
            print ("The report was not processed on the TNS "\
                   "because of the bad request(s).")

# Disable print
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore print
def enablePrint():
    sys.stdout.close()
    sys.stdout = old_stdout

# sending tsv or json report (at or class) and printing reply
def send_report(url, report, type_of_report):
    # sending report and checking response
    print ("Sending "+report+" to the TNS...")
    # choose which function to call
    if type_of_report=="tsv":
        response=send_tsv_report(url,report)
    else:
        response=send_json_report(url,report)
    response_check=check_response(response)
    # if report is sent
    if response_check==True:
        print ("The report was sent to the TNS.")
        # report response as json data
        json_data=response.json()
        # taking report id
        report_id=str(json_data['data']['report_id'])
        print ("Report ID = "+report_id)
        print ("")
        # sending report id to get reply of the report
        # and printing that reply
        # waiting for report to arrive before sending reply
        # for report id
        blockPrint()
        counter = 0
        while True:
            time.sleep(SLEEP_SEC)
            reply_response=reply(url,report_id)
            reply_res_check=check_response(reply_response)
            if reply_res_check!=False or counter >= LOOP_COUNTER:
                break
            counter += 1
        enablePrint()
        print_reply(url,report_id)
    else:
        print ("The report was not sent to the TNS.")

# uploading files and printing reply
def upload(url, list_of_files):
    # upload files and checking response
    print ("Uploading files on the TNS...")
    response=upload_files(url,list_of_files)
    response_check=check_response(response)
    # if files are uploaded
    if response_check==True:
        print ("The following files are uploaded on the TNS : ")
        # response as json data
        json_data=response.json()
        # list of uploaded files
        uploaded_files=json_data['data']
        for i in range(len(uploaded_files)):
            print ("filename : "+str(uploaded_files[i]))
    else:
        print ("Files are not uploaded on the TNS.")
    print ("\n")

###########################################################################################
###########################################################################################


# EXAMPLE

# Comment/Uncomment sections for testing the various examples:

"""
# ---------------------------------------------------
# upload files
list_of_filenames=["rel_file_1.png","rel_file_2.jpg",
                   "spectra_example.asci.txt",
                   "spectra_example.fits",
                   "frb_region_file_example.fits"]
upload(url_tns_api,list_of_filenames)
# ---------------------------------------------------
"""

"""
# ---------------------------------------------------
# send AT report
report_filename="json_at_report.txt"
report_type="json" 
# OR 
report_filename="bulk_tsv_at_report.txt"
report_type="tsv"
send_report(url_tns_api,report_filename,report_type)
# ---------------------------------------------------
"""

"""
# ---------------------------------------------------
# send Classification report
report_filename="json_classification_report.txt"
report_type="json"
# OR 
report_filename="bulk_tsv_classification_report.txt"
report_type="tsv"
send_report(url_tns_api,report_filename,report_type)
# ---------------------------------------------------
"""

"""
# ---------------------------------------------------
# send FRB report
report_filename="json_frb_report.txt"
report_type="json"
send_report(url_tns_api,report_filename,report_type)
# ---------------------------------------------------
"""

"""
# ---------------------------------------------------
# reply
id_report="62086"
print_reply(url_tns_api,id_report)
#---------------------------------------------------
"""



'\n# ---------------------------------------------------\n# reply\nid_report="62086"\nprint_reply(url_tns_api,id_report)\n#---------------------------------------------------\n'

<a class="anchor" id="report"></a>
# Report an astronomical transient from an ANTARES locus object

The next block of code shows how to use ANTARES locus information and create a report. The following piece of code will create an AT report in json format, and store in the local directory json_reports/for_sending/ .

In [3]:
from antares_client.search import get_by_id, get_by_ztf_object_id, search
import astropy.time as atime
import datetime
import requests


obj_id="ANT2021negc4"

locus=get_by_id(obj_id)

#sorting out filter of the lastest detection
if locus.alerts[-1].properties['ztf_fid']==1:
    filter_id="110"
if locus.alerts[-1].properties['ztf_fid']==2:
    filter_id="111"

#sorting out jd, diffmaglim, and filter of the lastest non-detection
for alert in locus.alerts:
    if alert.properties['ant_survey'] == 2:
            non_det_jd = alert.properties['ztf_jd']
            non_det_maglim = alert.properties['ztf_diffmaglim']
            if alert.properties['ztf_fid'] == 1:
                non_det_fid="110"
            if alert.properties['ztf_fid'] == 2:
                non_det_fid="111"
    
antares_data={
  "at_report": {
    "0": {
      "ra": {
        "value": str(locus.ra),
#        "error": "0.5",
        "units": "deg"
      },
      "dec": {
        "value": str(locus.dec),
#        "error": "0.5",
        "units": "deg"
      },
      "reporting_group_id": "115",
      "discovery_data_source_id": "48",
      "reporter": "Patrick Aleo (UIUC), Chien-Hsiu Lee (NSF's NOIRLab), Konstantin Malanchev (UIUC), Thomas Matheson (NSF's NOIRLab), Gautham Narayan (UIUC), Abhijit Saha (NSF's NOIRLab), Carlos Scheidegger (U. Arizona), Adam Scott (NSF's NOIRLab), Monika Soraisam (UIUC), Carl Stubens (NSF's NOIRLab), Nicholas Wolf (NSF's NOIRLab)",
      "discovery_datetime":str(locus.properties['newest_alert_observation_time']+2400000.5),
      "at_type": "1",
      "host_name": "",
      "host_redshift": "",
      "transient_redshift": "",
      "internal_name": str(locus.properties['ztf_object_id']),
      "internal_name_format": {
        "prefix": "prefixStr",
        "year_format": "YY",
        "postfix": "postfixStr"
      },
      "remarks": "Supernova candidate identified by the ANTARES anomaly detection filter. See locus info at: https://antares.noirlab.edu/loci/"+str(locus.locus_id),
      "proprietary_period_groups": [
        "115"
      ],
      "proprietary_period": {
        "proprietary_period_value": "0",
        "proprietary_period_units": "days"
      },
      "non_detection": {
        "obsdate": str(non_det_jd),
        "limiting_flux":str(non_det_maglim),
        "flux_units": "1",
        "filter_value": non_det_fid,
        "instrument_value": "196",
        "exptime": "60",
        "observer": "ZTF",
        "comments": "",
        "archiveid": "",
        "archival_remarks": ""
      },
      "photometry": {
        "photometry_group": {
          "0": {
            "obsdate":str(locus.alerts[-1].properties['ztf_jd']),
            "flux":str(locus.alerts[-1].properties['ztf_magpsf']),
            "flux_error":str(locus.alerts[-1].properties['ztf_sigmapsf']),
            "limiting_flux":str(locus.alerts[-1].properties['ztf_diffmaglim']),
            "flux_units": "1",
            "filter_value": filter_id,
            "instrument_value": "196",
            "exptime": "60",
            "observer": "ZTF",
            "comments": ""
          }
        }
      }
    }
  }
}

filename=obj_id+'_data.txt'

with open('json_reports_for_sending/'+filename, 'w') as outfile:
    json.dump(antares_data, outfile)

# send AT report
report_filename=filename
report_type="json" 

send_report(url_tns_api,report_filename,report_type)




Sending ANT2021negc4_data.txt to the TNS...
ID code = 200
ID message = OK
The report was sent to the TNS.
Report ID = 72765

Sending reply for the report id 72765 ...
ID code = 200
ID message = OK
The report was successfully processed on the TNS.

----------------------------------------------------------------------
Message ID = 100
Message = Transient object was inserted. Object name = 2021ltr
----------------------------------------------------------------------



<a class="anchor" id="bulkreport"></a>
# Bulk report from a list of ANTARES loci

The next block of code shows how to loop over a list of ANTARES loci and sending bulk report of them. The following piece of code will create AT reports in json format, and store them in the local directory json_reports/for_sending/ .

In [4]:
#https://antares.noirlab.edu/loci/ANT2021lzqbs 
#https://antares.noirlab.edu/loci/ANT2021mdzkk 
#https://antares.noirlab.edu/loci/ANT2021lztko 
#https://antares.noirlab.edu/loci/ANT2021l3w6y (follow up? Still rising!)
#https://antares.noirlab.edu/loci/ANT2021l2nrg (follow up? Still rising!)
#https://antares.noirlab.edu/loci/ANT2021malay (follow up? Kinda still rising!)
#https://antares.noirlab.edu/loci/ANT2021lny7a (high-z SN ; z=0.16)
#https://antares.noirlab.edu/loci/ANT2021l5e3i 
#https://antares.noirlab.edu/loci/ANT2021l5dvw 
#https://antares.noirlab.edu/loci/ANT2021lx7z2 
loci = ['ANT2021lzqbs','ANT2021mdzkk','ANT2021lztko','ANT2021l3w6y','ANT2021l2nrg','ANT2021malay','ANT2021lny7a','ANT2021l5e3i','ANT2021l5dvw','ANT2021lx7z2' ]

for obj_id in loci:
    locus=get_by_id(obj_id)
    
    #sorting out filter of the lastest detection
    if locus.alerts[-1].properties['ztf_fid']==1:
        filter_id="110"
    if locus.alerts[-1].properties['ztf_fid']==2:
        filter_id="111"

    #sorting out jd, diffmaglim, and filter of the lastest non-detection
    for alert in locus.alerts:
        if alert.properties['ant_survey'] == 2:
                non_det_jd = alert.properties['ztf_jd']
                non_det_maglim = alert.properties['ztf_diffmaglim']
                if alert.properties['ztf_fid'] == 1:
                    non_det_fid="110"
                if alert.properties['ztf_fid'] == 2:
                    non_det_fid="111"
    
    antares_data={
      "at_report": {
        "0": {
          "ra": {
            "value": str(locus.ra),
    #        "error": "0.5",
            "units": "deg"
          },
          "dec": {
            "value": str(locus.dec),
    #        "error": "0.5",
            "units": "deg"
          },
          "reporting_group_id": "115",
          "discovery_data_source_id": "48",
          "reporter": "Patrick Aleo (UIUC), Chien-Hsiu Lee (NSF's NOIRLab), Konstantin Malanchev (UIUC), Thomas Matheson (NSF's NOIRLab), Gautham Narayan (UIUC), Abhijit Saha (NSF's NOIRLab), Carlos Scheidegger (U. Arizona), Adam Scott (NSF's NOIRLab), Monika Soraisam (UIUC), Carl Stubens (NSF's NOIRLab), Nicholas Wolf (NSF's NOIRLab)",
          "discovery_datetime":str(locus.properties['newest_alert_observation_time']+2400000.5),
          "at_type": "1",
          "host_name": "",
          "host_redshift": "",
          "transient_redshift": "",
          "internal_name": str(locus.properties['ztf_object_id']),
          "internal_name_format": {
            "prefix": "prefixStr",
            "year_format": "YY",
            "postfix": "postfixStr"
          },
          "remarks": "Supernova candidate identified by the ANTARES anomaly detection filter. See locus info at: https://antares.noirlab.edu/loci/"+str(locus.locus_id),
          "proprietary_period_groups": [
            "115"
          ],
          "proprietary_period": {
            "proprietary_period_value": "0",
            "proprietary_period_units": "days"
          },
          "non_detection": {
            "obsdate": str(non_det_jd),
            "limiting_flux":str(non_det_maglim),
            "flux_units": "1",
            "filter_value": non_det_fid,
            "instrument_value": "196",
            "exptime": "60",
            "observer": "ZTF",
            "comments": "",
            "archiveid": "",
            "archival_remarks": ""
          },
          "photometry": {
            "photometry_group": {
              "0": {
                "obsdate":str(locus.alerts[-1].properties['ztf_jd']),
                "flux":str(locus.alerts[-1].properties['ztf_magpsf']),
                "flux_error":str(locus.alerts[-1].properties['ztf_sigmapsf']),
                "limiting_flux":str(locus.alerts[-1].properties['ztf_diffmaglim']),
                "flux_units": "1",
                "filter_value": filter_id,
                "instrument_value": "196",
                "exptime": "60",
                "observer": "ZTF",
                "comments": ""
              }
            }
          }
        }
      }
    }

    filename=obj_id+'_data.txt'

    with open('json_reports_for_sending/'+filename, 'w') as outfile:
        json.dump(antares_data, outfile)

    # send AT report
    report_filename=filename
    report_type="json" 
    
    send_report(url_tns_api,report_filename,report_type)




Sending ANT2021lzqbs_data.txt to the TNS...
ID code = 200
ID message = OK
The report was sent to the TNS.
Report ID = 72766

Sending reply for the report id 72766 ...
ID code = 200
ID message = OK
The report was successfully processed on the TNS.

----------------------------------------------------------------------
Message ID = 101
Message = Transient object exists. Object name = 2021ljj
Message ID = 124
Message = Object coordinates were updated according to those specified in this report.
----------------------------------------------------------------------

Sending ANT2021mdzkk_data.txt to the TNS...
ID code = 200
ID message = OK
The report was sent to the TNS.
Report ID = 72767

Sending reply for the report id 72767 ...
ID code = 400
ID message = Bad request
The report was not processed on the TNS because of the bad request(s).
Sending ANT2021lztko_data.txt to the TNS...
ID code = 200
ID message = OK
The report was sent to the TNS.
Report ID = 72768

Sending reply for the report 